In [1]:
import sys
sys.path.insert(0, './../')
import bbdc2021 as bbdc

In [2]:
pipe_param = {'data_folder': './../data/',
                  'wav_files_folder': 'final_pre_dataset', # fft parameter
                  'window_length': 1024, 'window_overlap': 523,
                  'band_size': 4, 'sample_rate': 16000,
                  'dev_csv': 'dev-labels.csv', # filelisten, falls man doch auf kleinerem probieren möchte
                  'eval_csv': 'challenge_filelist_dummy.csv',
                  'scaling': 'no', # 'standard', 'minmax' oder 'no' wenn kein scaling sein soll
                  'test_split_range': (9000, -1),
                  'prediction_path': './', # wo prediction csvs gespeichert werden sollen
                  'post_processing': 'fill', # 'fill' is das von Jannes
                  'submission_file_path': './challenge_submission.csv'}
unet_param = {'channels': [32, 64, 64],
              'lessParameter': True,
              'val_split_fraction': 1/9, # 1/9 damit man 1000 von den 9000 abzweigt
              'loss': 'categorical_crossentropy', # bbdc.dice_loss möglich
              'learning_rate': 0.001,
              'batch_size': 15,
              'epochs': 100,
              'model_save_path': './',
              'load_path': None} # Wenn trainiert werden soll: None; Submission 1: 'model9561.h5'

In [3]:
def pipeline_u_net_1(pipe_param, model_param):
    """Current u_net pipeline from loading data to prediction.
    Takes paramater dictionary as argument"""
    # load
    x_dev, y_dev, timep, filelist_dev, x_ch, filelist_ch = bbdc.loading_block1(pipe_param)
    # split (tv=train and val set)
    x_tv, x_test, y_tv, y_test, filelist_test = bbdc.split_block1(x_dev, y_dev, timep, filelist_dev, pipe_param)
    # model fit (or load)
    history, model = bbdc.model_block1_unet(x_tv, y_tv, unet_param)
    # evaluate
    bbdc.evaluation_block1(x_test, y_test, timep, filelist_test, model, pipe_param)
    # post processing and its evaluation
    pp_pred_test, pp_pred_ch = bbdc.postprocessing_with_evaluation_block1(x_test, y_test, timep, filelist_test,
                                                                         x_ch, filelist_ch, model, pipe_param)
    # challenge prediction
    bbdc.challenge_prediction_block1(pp_pred_ch, timep, filelist_ch, pipe_param)
    return history

In [4]:
# cross validation 10 fold
test_index_list = [(0, 1000), (1000, 2000), (2000, 3000), (3000, 4000), (4000, 5000),
                   (5000, 6000), (6000, 7000), (7000, 8000), (8000, 9000), (9000, -1)]
for indices in test_index_list:
    pipe_param['test_split_range'] = indices
    print('Test Split:', indices)
    pipeline_u_net_1(pipe_param, unet_param)
    print('\n\n\n')

Test Split: (0, 1000)
Csv from wav files already existend. Skipping calc_fft.
Scaled numpy files already existend. Skipping scaling and load_data function.
Splitting test set at indices 0 to 1000 from dev set.
Tensorflow version: 2.4.1
Epoch 1/100
534/534 [==============================] - 42s 60ms/step - loss: 1.5089 - mae: 0.0907 - accuracy: 0.5989 - val_loss: 1.4308 - val_mae: 0.0560 - val_accuracy: 0.6626

Epoch 00001: val_loss improved from inf to 1.43076, saving model to model.h5
Epoch 2/100
534/534 [==============================] - 31s 57ms/step - loss: 0.7957 - mae: 0.0510 - accuracy: 0.7624 - val_loss: 3.7530 - val_mae: 0.1286 - val_accuracy: 0.1814

Epoch 00002: val_loss did not improve from 1.43076
Epoch 3/100
534/534 [==============================] - 31s 58ms/step - loss: 0.6246 - mae: 0.0411 - accuracy: 0.8138 - val_loss: 5.7198 - val_mae: 0.1355 - val_accuracy: 0.1236

Epoch 00003: val_loss did not improve from 1.43076
Epoch 4/100
534/534 [==============================

534/534 [==============================] - 30s 56ms/step - loss: 0.2063 - mae: 0.0146 - accuracy: 0.9362 - val_loss: 1.4232 - val_mae: 0.0643 - val_accuracy: 0.6165

Epoch 00035: val_loss did not improve from 0.90918
Epoch 36/100
534/534 [==============================] - 30s 56ms/step - loss: 0.2035 - mae: 0.0145 - accuracy: 0.9371 - val_loss: 2.3200 - val_mae: 0.0887 - val_accuracy: 0.4316

Epoch 00036: val_loss did not improve from 0.90918
Epoch 37/100
534/534 [==============================] - 30s 56ms/step - loss: 0.2008 - mae: 0.0143 - accuracy: 0.9379 - val_loss: 1.7606 - val_mae: 0.0686 - val_accuracy: 0.5876

Epoch 00037: val_loss did not improve from 0.90918
Epoch 38/100
534/534 [==============================] - 30s 56ms/step - loss: 0.2061 - mae: 0.0146 - accuracy: 0.9361 - val_loss: 1.1366 - val_mae: 0.0556 - val_accuracy: 0.6836

Epoch 00038: val_loss did not improve from 0.90918
Epoch 39/100
534/534 [==============================] - 30s 56ms/step - loss: 0.2040 - mae: 0

534/534 [==============================] - 29s 55ms/step - loss: 0.1594 - mae: 0.0115 - accuracy: 0.9501 - val_loss: 0.5845 - val_mae: 0.0309 - val_accuracy: 0.8355

Epoch 00070: val_loss did not improve from 0.52067
Epoch 71/100
534/534 [==============================] - 29s 55ms/step - loss: 0.1582 - mae: 0.0114 - accuracy: 0.9508 - val_loss: 0.3972 - val_mae: 0.0223 - val_accuracy: 0.8858

Epoch 00071: val_loss improved from 0.52067 to 0.39721, saving model to model.h5
Epoch 72/100
534/534 [==============================] - 29s 55ms/step - loss: 0.1552 - mae: 0.0113 - accuracy: 0.9513 - val_loss: 0.4003 - val_mae: 0.0210 - val_accuracy: 0.8915

Epoch 00072: val_loss did not improve from 0.39721
Epoch 73/100
534/534 [==============================] - 29s 55ms/step - loss: 0.1564 - mae: 0.0114 - accuracy: 0.9502 - val_loss: 1.0715 - val_mae: 0.0514 - val_accuracy: 0.6987

Epoch 00073: val_loss did not improve from 0.39721
Epoch 74/100
534/534 [==============================] - 29s 55m

/home/alex/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alex/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Submission file saved at ./challenge_submission.csv




Test Split: (1000, 2000)
Csv from wav files already existend. Skipping calc_fft.
Scaled numpy files already existend. Skipping scaling and load_data function.
Splitting test set at indices 1000 to 2000 from dev set.
Tensorflow version: 2.4.1
Epoch 1/100
534/534 [==============================] - 52s 59ms/step - loss: 1.4333 - mae: 0.0858 - accuracy: 0.6021 - val_loss: 1.6326 - val_mae: 0.0626 - val_accuracy: 0.6270

Epoch 00001: val_loss improved from inf to 1.63259, saving model to model.h5
Epoch 2/100
534/534 [==============================] - 29s 55ms/step - loss: 0.8067 - mae: 0.0518 - accuracy: 0.7576 - val_loss: 1.3648 - val_mae: 0.0718 - val_accuracy: 0.6120

Epoch 00002: val_loss improved from 1.63259 to 1.36484, saving model to model.h5
Epoch 3/100
534/534 [==============================] - 29s 55ms/step - loss: 0.6565 - mae: 0.0430 - accuracy: 0.8010 - val_loss: 1.2997 - val_mae: 0.0739 - val_accuracy: 0.5889

Epoch 0000

...

Submission file saved at ./challenge_submission.csv




Test Split: (9000, -1)
Csv from wav files already existend. Skipping calc_fft.
Scaled numpy files already existend. Skipping scaling and load_data function.
Splitting test set at indices 9000 to -1 from dev set.
Tensorflow version: 2.4.1
Epoch 1/100
593/593 [==============================] - 50s 63ms/step - loss: 1.3822 - mae: 0.0834 - accuracy: 0.6348 - val_loss: 2.1819 - val_mae: 0.1154 - val_accuracy: 0.3178

Epoch 00001: val_loss improved from inf to 2.18191, saving model to model.h5
Epoch 2/100
593/593 [==============================] - 33s 55ms/step - loss: 0.7366 - mae: 0.0473 - accuracy: 0.7818 - val_loss: 2.7610 - val_mae: 0.1131 - val_accuracy: 0.2930

Epoch 00002: val_loss did not improve from 2.18191
Epoch 3/100
593/593 [==============================] - 33s 55ms/step - loss: 0.6032 - mae: 0.0397 - accuracy: 0.8197 - val_loss: 2.6566 - val_mae: 0.1136 - val_accuracy: 0.2711

Epoch 00003: val_loss did not improve from 2

593/593 [==============================] - 32s 55ms/step - loss: 0.2022 - mae: 0.0144 - accuracy: 0.9375 - val_loss: 1.7528 - val_mae: 0.0716 - val_accuracy: 0.5567

Epoch 00035: val_loss did not improve from 0.51862
Epoch 36/100
593/593 [==============================] - 33s 55ms/step - loss: 0.1949 - mae: 0.0139 - accuracy: 0.9394 - val_loss: 2.5877 - val_mae: 0.0813 - val_accuracy: 0.4896

Epoch 00036: val_loss did not improve from 0.51862
Epoch 37/100
593/593 [==============================] - 33s 55ms/step - loss: 0.1957 - mae: 0.0139 - accuracy: 0.9394 - val_loss: 1.0295 - val_mae: 0.0469 - val_accuracy: 0.7293

Epoch 00037: val_loss did not improve from 0.51862
Epoch 38/100
593/593 [==============================] - 32s 55ms/step - loss: 0.1914 - mae: 0.0138 - accuracy: 0.9406 - val_loss: 0.9295 - val_mae: 0.0396 - val_accuracy: 0.7811

Epoch 00038: val_loss did not improve from 0.51862
Epoch 39/100
593/593 [==============================] - 33s 55ms/step - loss: 0.1949 - mae: 0

593/593 [==============================] - 32s 55ms/step - loss: 0.1557 - mae: 0.0113 - accuracy: 0.9511 - val_loss: 0.8022 - val_mae: 0.0396 - val_accuracy: 0.7806

Epoch 00070: val_loss did not improve from 0.46945
Epoch 71/100
593/593 [==============================] - 32s 55ms/step - loss: 0.1511 - mae: 0.0109 - accuracy: 0.9525 - val_loss: 0.4628 - val_mae: 0.0236 - val_accuracy: 0.8738

Epoch 00071: val_loss improved from 0.46945 to 0.46284, saving model to model.h5
Epoch 72/100
593/593 [==============================] - 33s 55ms/step - loss: 0.1506 - mae: 0.0109 - accuracy: 0.9527 - val_loss: 1.3340 - val_mae: 0.0601 - val_accuracy: 0.6383

Epoch 00072: val_loss did not improve from 0.46284
Epoch 73/100
593/593 [==============================] - 32s 55ms/step - loss: 0.1463 - mae: 0.0107 - accuracy: 0.9536 - val_loss: 1.9542 - val_mae: 0.0643 - val_accuracy: 0.6041

Epoch 00073: val_loss did not improve from 0.46284
Epoch 74/100
593/593 [==============================] - 33s 55m

/home/alex/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alex/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Submission file saved at ./challenge_submission.csv






In [11]:
fold = 10
print('Loss mean',                sum([2.22, 5.63, 2.43, 0.94, 0.45, 2.62, 2.05, 0.74, 0.44, 0.81])/fold)
print('MAE mean',                 sum([0.07, 0.11, 0.07, 0.04, 0.02, 0.08, 0.08, 0.04, 0.02, 0.03])/fold)
print('Accuracy mean',            sum([0.55, 0.28, 0.52, 0.75, 0.87, 0.46, 0.52, 0.79, 0.89, 0.82])/fold)
print('Model PSDS mean',          sum([0.07, 0.11, 0.09, 0.12, 0.11, 0.05, 0.04, 0.06, 0.17, 0.15])/fold)
print('Post processed PSDS mean', sum([0.50, 0.25, 0.33, 0.55, 0.70, 0.38, 0.31, 0.59, 0.72, 0.51])/fold)

Loss mean 1.8329999999999997
MAE mean 0.05600000000000001
Accuracy mean 0.645
Model PSDS mean 0.09700000000000002
Post processed PSDS mean 0.484
